In [2]:

import torch
print("torch:", torch.__version__, "cuda:", torch.version.cuda)
print("CUDA available:", torch.cuda.is_available())
print("Device:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "CPU")


torch: 2.8.0+cu128 cuda: 12.8
CUDA available: True
Device: NVIDIA GeForce RTX 3080


In [1]:
from transformers import pipeline, VideoMAEForVideoClassification
import accelerate
import scipy
import librosa as lr
import soundfile as sf
import numpy as np
import pandas as pd
import os
import tensorflow as tf
from datasets import load_dataset
from decord import VideoReader
from decord import cpu, gpu
import kagglehub


/mnt/c/Users/Admin/Documents/GitHub/Music-AI/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-10-21 18:38:54.137214: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-10-21 18:38:54.667209: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-10-21 18:39:10.120673: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You

# Video emotion detection
- training a model to detect the emotion present in a video or scene
- the scene may have people in it or maybe it could convey a mood based on the color
- I selected VideoMAE because it is very effcient with data meaning that it can be used when i dont have enough video data


In [3]:
# Load VideoMAE model
MAE_model = VideoMAEForVideoClassification.from_pretrained("MCG-NJU/videomae-base-finetuned-kinetics", attn_implementation="sdpa", dtype=torch.float16)
print("VideoMAE is loaded")

VideoMAE is loaded


# Preprocessing the Video data for emotion detection


In [10]:
# using the `kagglehub` library to download the MSR-VTT dataset
path = kagglehub.dataset_download("vishnutheepb/msrvtt")
msrvtt_videos_path = "/home/zach/.cache/kagglehub/datasets/vishnutheepb/msrvtt/versions/1/TrainValVideo"
print("Path to dataset files:", path)

100%|██████████| 4.26G/4.26G [01:48<00:00, 42.1MB/s]

Extracting files...


Path to dataset files: /home/zach/.cache/kagglehub/datasets/vishnutheepb/msrvtt/versions/1


In [14]:
videos_list = os.listdir(msrvtt_videos_path)
print("Number of videos in MSR-VTT dataset:", len(videos_list))
print("Sample video file:", videos_list[:5])
if isinstance(videos_list, list):
    print("my_variable is a list.")

Number of videos in MSR-VTT dataset: 7010
Sample video file: ['video1250.mp4', 'video4543.mp4', 'video6671.mp4', 'video1990.mp4', 'video6803.mp4']
my_variable is a list.


In [21]:
# getting all of the file paths to add to a dataframe
file_paths = [os.path.join(msrvtt_videos_path, f) for f in videos_list]
print("Sample file paths:", file_paths[:5])

# turn the video paths into a dataframe
path_df = pd.DataFrame({
    "file_path": file_paths,
})
path_df.head()

# extract the video names from the file paths
path_df["video_name"] = path_df["file_path"].apply(os.path.basename)

Sample file paths: ['/home/zach/.cache/kagglehub/datasets/vishnutheepb/msrvtt/versions/1/TrainValVideo/video1250.mp4', '/home/zach/.cache/kagglehub/datasets/vishnutheepb/msrvtt/versions/1/TrainValVideo/video4543.mp4', '/home/zach/.cache/kagglehub/datasets/vishnutheepb/msrvtt/versions/1/TrainValVideo/video6671.mp4', '/home/zach/.cache/kagglehub/datasets/vishnutheepb/msrvtt/versions/1/TrainValVideo/video1990.mp4', '/home/zach/.cache/kagglehub/datasets/vishnutheepb/msrvtt/versions/1/TrainValVideo/video6803.mp4']


In [16]:
# loading MSR-VTT dataset with the train_7k split and convert it to a dataframe
msrvtt_data_train = load_dataset("friedrichor/MSR-VTT", name="train_7k")
msrvtt_data_train_df=msrvtt_data_train['train'].to_pandas()
msrvtt_data_train_df.head()
print("MSR-VTT training dataset loaded with", len(msrvtt_data_train_df), "entries.")

MSR-VTT training dataset loaded with 7010 entries.


In [25]:
# merging the dataframes on the video file names
full_msrvtt_df = msrvtt_data_train_df.merge(path_df[["video_name" ,"file_path"]], on="Video", how="left")

KeyError: 'Video'

### Reading in the videos from the dataset

In [ ]:



    
### Reading in the videos from the dataset
def video_clipping(video_path, fps=2,max_frames=32):
    """Extract frames from a video file."""
    
    # vr is used create a VideoReader object to read the video file which allows for access to individual frames
    vr = VideoReader(video_path, ctx=cpu(0))
    
    # idx is used to create a list of frame indices to be extracted from the video or sampled uniformly across the video's duration
    idx = list(range(0, len(vr), max(1, len(vr)//max_frames)))[:max_frames]
    
    # frames is used to extract the frames from the video at the specified indices and convert them to a numpy array
    frames = vr.get_batch(idx).asnumpy()   # (T, H, W, 3) uint8
    
    return frames

# Testing the video_clipping function
clip = video_clipping(row["video"])


TypeError: expected str, bytes or os.PathLike object, not list